<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# Langgraph - Prompt Chaining

This notebook demonstrates how to use prompt chaining with LangGraph to build a multi-step email assistant. The assistant guides the writing process through three distinct stages:

- Generating an outline based on subject and bullet points

- Writing the initial draft using the outline and desired tone

- Refining tone if needed to match the specified style

This approach enables fine-grained control over the content generation process by decomposing the task into logical steps. Each stage in the graph is handled by a separate node, enabling targeted prompting, intermediate outputs, and conditional logic.

In addition, the entire workflow is instrumented with Phoenix, which provides OpenTelemetry-powered tracing and debugging. You can inspect each step’s inputs, outputs, and transitions directly in the Phoenix UI to identify bottlenecks or missteps in generation.

In [ ]:
!pip install langgraph langchain langchain_community "arize-phoenix" arize-phoenix-otel openinference-instrumentation-langchain

This is a template for prompt chaining with LangGraph. It is an email writer, with 3 steps: writing an outline, writing the email, and refining tone.

In [ ]:
from langgraph.graph import StateGraph, START, END
import os, getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


# Configure Phoenix Tracing

Make sure you go to https://app.phoenix.arize.com/ and generate an API key. This will allow you to trace your Langgraph application with Phoenix.

In [ ]:
PHOENIX_API_KEY = getpass.getpass("Phoenix API Key:")
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

Phoenix API Key:··········


In [ ]:
from phoenix.otel import register

tracer_provider = register(
  project_name="Prompt Chaining",
  auto_instrument=True
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: Prompt Chaining
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



# LLM of choice

In [ ]:
from typing_extensions import TypedDict, Literal
from IPython.display import Image, display

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)


# Defining Graph State

The *EmailState* defines the shared memory for our email-writing agent. Each field represents the evolving state of the email — starting from the user’s initial inputs (subject, notes, tone) and gradually building up through the stages of outline generation, drafting, and final tone refinement. This state dictionary is passed between nodes to ensure context is maintained and updated incrementally throughout the workflow.

In [ ]:
class EmailState(TypedDict, total=False):
    subject: str
    bullet_points: str           # raw user notes
    desired_tone: str            # "formal", "friendly", etc.
    outline: str                 # result of node 1
    draft_email: str             # result of node 2
    final_email: str             # after tone reformer (if needed)


# Step-by-Step Prompt Chain: Outline → Draft → Tone Check

This workflow chains multiple LLM calls to transform raw notes into a polished email:

**generate_outline**: Converts user bullet points into a structured outline.

**write_email**: Expands the outline into a complete email draft using the desired tone.

**tone_gate**: Checks if the draft meets the requested tone using a lightweight LLM classification.

**reform_tone**: If the tone doesn't match, this node rewrites the draft while preserving the content.

Each node is modular, enabling targeted debugging and reuse across different tasks or formats. This multi-step refinement mirrors human drafting processes and produces higher-quality outputs.

In [ ]:
def generate_outline(state: EmailState) -> EmailState:
    """LLM call 1 – produce an outline from bullet points."""
    prompt = (
        "Create a concise outline for an email.\n"
        f"Subject: {state['subject']}\n"
        f"Bullet points:\n{state['bullet_points']}\n"
        "Return the outline as numbered points."
    )
    outline = llm.invoke(prompt).content
    return {"outline": outline}

def write_email(state: EmailState) -> EmailState:
    """LLM call 2 – write the email from the outline."""
    prompt = (
        f"Write a complete email using this outline:\n{state['outline']}\n\n"
        f"Tone: {state['desired_tone']}\n"
        "Start with a greeting, respect professional formatting, and keep it concise."
    )
    email = llm.invoke(prompt).content
    return {"draft_email": email}

def tone_gate(state: EmailState) -> Literal["Pass", "Fail"]:
    """
    Gate – quick heuristic:
      Pass  → email already includes the required tone keyword.
      Fail  → otherwise (we’ll ask another LLM call to adjust).
    """
    tone_keyword = state["desired_tone"].lower()
    prompt = (
        f"Check whether the following email matches the desired tone {state['desired_tone']}:\n\n"
        f"{state['draft_email']}\n"
        f"If it does, return 'Pass'. Otherwise, return 'Fail'."
    )
    return llm.invoke(prompt).content.strip(
    )

def reform_tone(state: EmailState) -> EmailState:
    """LLM call 3 – rewrite the email to fit the desired tone."""
    prompt = (
        f"Reform the following email so it has a {state['desired_tone']} tone.\n\n"
        f"EMAIL:\n{state['draft_email']}\n\n"
        "Keep content the same but adjust phrasing, word choice, and sign‑off."
    )
    final_email = llm.invoke(prompt).content
    return {"final_email": final_email}


# Compiling the Email Prompt Chain with LangGraph

Here we assemble the full LangGraph that represents our email generation pipeline. The graph begins at the outline_generator, moves to the email_writer, and conditionally routes to tone_reformer only if the tone check fails. This structure demonstrates the prompt chaining pattern with a dynamic control flow—adapting based on the model’s output. Once compiled, this graph can be invoked on user input and traced using Phoenix for debugging or optimization.

In [ ]:
graph = StateGraph(EmailState)

graph.add_node("outline_generator", generate_outline)
graph.add_node("email_writer", write_email)
graph.add_node("tone_reformer", reform_tone)

# edges
graph.add_edge(START, "outline_generator")
graph.add_edge("outline_generator", "email_writer")
graph.add_conditional_edges(
    "email_writer",
    tone_gate,
    {"Pass": END, "Fail": "tone_reformer"},
)
graph.add_edge("tone_reformer", END)

email_chain = graph.compile()

# ────────────────────────────────────────────────
# 4. Visualize & run once
# ────────────────────────────────────────────────
display(Image(email_chain.get_graph().draw_mermaid_png()))


# Example Usage

In [ ]:
initial_state = email_chain.invoke(
    {
        "subject": "Quarterly Sales Recap & Next Steps",
        "bullet_points": "- Q1 revenue up 18%\n- Need feedback on new pricing tiers\n- Reminder: submit pipeline forecasts by Friday",
        "desired_tone": "friendly",
    }
)

print("\n========== EMAIL ==========")
print(initial_state.get("final_email", initial_state["draft_email"]))


# Make sure to view your traces in Phoenix!